<a href="https://colab.research.google.com/github/Hitomiblood/ProjectsDataScienceMaster/blob/main/Proyecto_TFM/Models/MotileDetection/PostBestModel/ExtractingROIFromVideoListComplete.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Creando procesamiento de Frames para extracción de secciones de espermatozoides móviles en un conjunto de videos.

In [1]:
%%capture
!pip install ultralytics

In [2]:
%%capture
!wget https://www.dropbox.com/scl/fi/maqbuw2e0s7ywa63dofun/bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt&st=04t3nonx&dl=0
!mv bestSmall1920.pt?rlkey=n0pqsnjgycak6v7jx3qzhladt bestSmall1920.pt

In [3]:
%%capture
# Descargar el archivo ZIP desde la URL
!wget https://www.dropbox.com/scl/fi/vnyl0lbmzarkfm7jrt88h/Videos.zip?rlkey=4i3y4so9xnomins88tpgdls0z&st=mgjlmjb3&dl=0

# Crear la carpeta "Videos" si no existe
!mkdir -p Videos

!mv Videos.zip?rlkey=4i3y4so9xnomins88tpgdls0z Videos.zip

# Descomprimir el archivo ZIP en la carpeta "Videos"
!unzip -o Videos.zip -d Videos


In [7]:
!rm Videos/acA1920-155um__22949301__20200710_094124535.avi
!rm Videos/acA1920-155um__22949301__20210625_105958792.avi
!rm Videos/acA1920-155um__22949301__20220526_124553810.avi
!rm Videos/acA1920-155um__22949301__20220526_160756867.avi
!rm Videos/acA1920-155um__22949301__20210628_094625267.avi
!rm Videos/acA1920-155um__22949301__20210628_114549599.avi
!rm Videos/Suppl_1.avi


In [4]:
from ultralytics import YOLO
# Load a pretrained YOLO model (recommended for training)
model = YOLO('bestSmall1920.pt')

In [8]:
import cv2
import os

def extraer_frames_video(video_path):
    # Cargar el video
    cap = cv2.VideoCapture(video_path)

    # Verificar si el video se ha abierto correctamente
    if not cap.isOpened():
        print(f"Error al abrir el video: {video_path}")
        return None

    frames = []
    frame_count = 0

    # Leer frames del video
    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frames.append(frame)
        frame_count += 1

    # Liberar el objeto VideoCapture
    cap.release()

    print(f"\n Total de frames extraídos del video {video_path}: {frame_count}")

    return frames

def recortar_y_guardar_imagenes(orig_img, result, output_folder, object_counter):
    # Aumentar el tamaño de las imágenes recolectadas en 30 píxeles por lado
    margin = 30

    # Crear la carpeta de salida si no existe
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)

    # Obtener las cajas y los IDs de seguimiento
    boxes = result[0].boxes.xyxyn.cpu()

    # Dimensiones de la imagen original
    img_height, img_width = orig_img.shape[:2]

    # Iterar sobre las cajas
    for box in boxes:
        # Desnormalizar las coordenadas de la caja
        left, top, right, bottom = box
        left = int(left * img_width)
        top = int(top * img_height)
        right = int(right * img_width)
        bottom = int(bottom * img_height)

        # Recortar la región de interés (ROI) de la imagen original
        roi = orig_img[max(top - margin, 0):min(bottom + margin, img_height),
                        max(left - margin, 0):min(right + margin, img_width)]

        # Guardar la región de interés como una imagen separada
        cv2.imwrite(os.path.join(output_folder, f"objeto_{object_counter}.jpg"), roi)
        object_counter += 1

    return object_counter

def procesar_videos(carpetas_videos, modelo, output_folder):
    object_counter = 0  # Contador global para los objetos detectados

    for carpeta in carpetas_videos:
        # Obtener la lista de archivos en la carpeta de video
        archivos_video = [archivo for archivo in os.listdir(carpeta) if archivo.endswith(".mp4") or archivo.endswith(".avi")]

        for archivo in archivos_video:
            video_path = os.path.join(carpeta, archivo)
            # Extraer todos los frames del video
            frames = extraer_frames_video(video_path)

            if frames is not None:
                # Procesar cada frame
                for frame_number, frame in enumerate(frames):
                    # Pasar el frame por el modelo
                    results = modelo(frame)
                    # Recortar y guardar las imágenes
                    object_counter = recortar_y_guardar_imagenes(frame, results, output_folder, object_counter)

    print(f"Todas las imágenes recortadas se han guardado exitosamente en {output_folder}")


In [9]:
# Lista de carpetas que contienen videos
carpetas_videos = ["Videos/"]

# Carpeta donde se guardarán las imágenes recortadas
carpeta_objetos_detectados = "objetos_detectados"

# Llamar a la función para procesar los videos
procesar_videos(carpetas_videos, model, carpeta_objetos_detectados)


 Total de frames extraídos del video Videos/acA1920-155um__22949301__20210628_114637792.avi: 59

0: 1216x1920 20 Movils, 3711.1ms
Speed: 49.1ms preprocess, 3711.1ms inference, 28.9ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 19 Movils, 3399.4ms
Speed: 14.7ms preprocess, 3399.4ms inference, 1.5ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 19 Movils, 4055.7ms
Speed: 14.2ms preprocess, 4055.7ms inference, 2.2ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 19 Movils, 3864.6ms
Speed: 16.4ms preprocess, 3864.6ms inference, 1.5ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 19 Movils, 3387.8ms
Speed: 15.5ms preprocess, 3387.8ms inference, 1.5ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 20 Movils, 3337.5ms
Speed: 14.0ms preprocess, 3337.5ms inference, 1.6ms postprocess per image at shape (1, 3, 1216, 1920)

0: 1216x1920 19 Movils, 4609.0ms
Speed: 14.5ms preprocess, 4609.0ms inference, 1.5ms 

In [10]:
import shutil
archivo = shutil.make_archive('objetos_detectados', 'zip','/content/objetos_detectados')